Data Source: https://www.kaggle.com/snap/amazon-fine-food-reviews

In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")


import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix

from sklearn.metrics import roc_curve, auc

import re
import string
from nltk.corpus import stopwords

from tqdm import tqdm
import os

In [2]:
# using SQLite Table to read data.
con = sqlite3.connect('database.sqlite') 

# filtering only positive and negative reviews i.e. 
# not taking into consideration those reviews with Score=3

filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3 LIMIT 50000""", con) 

# Give reviews with Score>3 a positive rating(1), and reviews with a score<3 a negative rating(0).
def partition(x):
    if x < 3:
        return 0
    return 1

#changing reviews with score less than 3 to be positive and vice-versa
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative
print("Number of data points in our data", filtered_data.shape)
filtered_data.head(3)

Number of data points in our data (50000, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


In [3]:
filtered_data['Score'].value_counts(normalize=True)

1    0.83576
0    0.16424
Name: Score, dtype: float64

In [4]:
display = pd.read_sql_query("""
SELECT UserId, ProductId, ProfileName, Time, Score, Text, COUNT(*)
FROM Reviews
GROUP BY UserId
HAVING COUNT(*)>1
""", con)

In [5]:
print(display.shape)
display.head()

(80668, 7)


,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
0,#oc-R115TNMSPFT9I7,B005ZBZLT4,Breyton,1331510400,2,Overall its just OK when considering the price...,2
1,#oc-R11D9D7SHXIJB9,B005HG9ESG,"Louis E. Emory ""hoppy""",1342396800,5,"My wife has recurring extreme muscle spasms, u...",3
2,#oc-R11DNU2NBKQ23Z,B005ZBZLT4,Kim Cieszykowski,1348531200,1,This coffee is horrible and unfortunately not ...,2
3,#oc-R11O5J5ZVQE25C,B005HG9ESG,Penguin Chick,1346889600,5,This will be the bottle that you grab from the...,3
4,#oc-R12KPBODL2B5ZD,B007OSBEV0,Christopher P. Presta,1348617600,1,I didnt like this coffee. Instead of telling y...,2


In [6]:
#It was inferred after analysis that reviews with same parameters other than ProductId belonged to the same product,
#just having different flavour or quantity.
#Hence in order to reduce redundancy it was decided to eliminate the rows having same parameters.

In [7]:
sorted_data=filtered_data.sort_values('Id', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [8]:
#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(46072, 10)

In [9]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [10]:
final['Text'] = final['Summary'].astype(str) + "\n" + \
                 final['Text'].astype(str)

In [11]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [12]:
# https://gist.github.com/sebleier/554280
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [13]:
from tqdm import tqdm
from bs4 import BeautifulSoup

preprocessed_reviews = []
# tqdm is for printing the status bar
for sentence in tqdm(final['Text'].values):
    sentence = re.sub(r"http\S+", "", sentence)
    sentence = BeautifulSoup(sentence, 'lxml').get_text()
    sentence = decontracted(sentence)
    sentence = re.sub("\S*\d\S*", "", sentence).strip()
    sentence = re.sub('[^A-Za-z]+', ' ', sentence)
    # https://gist.github.com/sebleier/554280
    sentence = ' '.join(e.lower() for e in sentence.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentence.strip())

100%|██████████| 46071/46071 [00:36<00:00, 1271.02it/s]


In [14]:
final['Text']=preprocessed_reviews

In [15]:
final.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,good quality dog food bought several vitality ...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,not advertised product arrived labeled jumbo s...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",delight says confection around centuries light...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,cough medicine looking secret ingredient robit...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,great taffy great taffy great price wide assor...


## Lemmatization or Stemming

In [16]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
lem=WordNetLemmatizer()
por=PorterStemmer()

In [17]:
tokens=[word_tokenize(word) for word in final['Text']]

In [18]:
stemmed_tokens=[[lem.lemmatize(word) for word in sentence] for sentence in tokens]

In [19]:
tokens[0]

['good',
 'quality',
 'dog',
 'food',
 'bought',
 'several',
 'vitality',
 'canned',
 'dog',
 'food',
 'products',
 'found',
 'good',
 'quality',
 'product',
 'looks',
 'like',
 'stew',
 'processed',
 'meat',
 'smells',
 'better',
 'labrador',
 'finicky',
 'appreciates',
 'product',
 'better']

In [20]:
stemmed_tokens[0]

['good',
 'quality',
 'dog',
 'food',
 'bought',
 'several',
 'vitality',
 'canned',
 'dog',
 'food',
 'product',
 'found',
 'good',
 'quality',
 'product',
 'look',
 'like',
 'stew',
 'processed',
 'meat',
 'smell',
 'better',
 'labrador',
 'finicky',
 'appreciates',
 'product',
 'better']

In [21]:
ab=[" ".join(i) for i in stemmed_tokens]

In [22]:
final['Text']=ab

In [23]:
final.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,good quality dog food bought several vitality ...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,not advertised product arrived labeled jumbo s...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",delight say confection around century light pi...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,cough medicine looking secret ingredient robit...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,great taffy great taffy great price wide assor...


In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(final["Text"], final['Score'], random_state = 0,shuffle=False)

In [25]:
#bi-gram, tri-gram and n-gram

#removing stop words like "not" should be avoided before building n-grams

count_vect = CountVectorizer(ngram_range=(1,2),max_features=5000)
final_bigram_counts = count_vect.fit_transform(X_train)
print("the shape of out text BOW vectorizer ",final_bigram_counts.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_bigram_counts.get_shape()[1])

the shape of out text BOW vectorizer  (34553, 5000)
the number of unique words including both unigrams and bigrams  5000


In [26]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2),max_features=5000)

final_tf_idf = tf_idf_vect.fit_transform(X_train)
print("the shape of out text TFIDF vectorizer ",final_tf_idf.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_tf_idf.get_shape()[1])

the shape of out text TFIDF vectorizer  (34553, 5000)
the number of unique words including both unigrams and bigrams  5000


In [27]:
# Readying test data

In [28]:
test_count = count_vect.transform(X_test)
test_tf = tf_idf_vect.transform(X_test)

In [29]:
test_count.get_shape()

(11518, 5000)

In [30]:
test_tf.get_shape()

(11518, 5000)

In [31]:
# Modeling

In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

from sklearn.metrics import roc_auc_score

In [33]:
def get_auc_score(X_train,y_train,classifier,vectorizer):
    cls=classifier()
    scores=cross_val_score(cls,X_train,y_train,scoring='roc_auc',cv=5)
    if vectorizer==count_vect:
        print('For countvectorizer:')
    else:
        print('For tf_idf')
    print('AUC: for ',classifier,' ', scores.mean())

In [34]:
get_auc_score(final_bigram_counts,y_train,LogisticRegression,count_vect)
get_auc_score(final_tf_idf,y_train,LogisticRegression,tf_idf_vect)

For countvectorizer:
AUC: for  <class 'sklearn.linear_model._logistic.LogisticRegression'>   0.9516442585994872
For tf_idf
AUC: for  <class 'sklearn.linear_model._logistic.LogisticRegression'>   0.9671704231545573


In [35]:
get_auc_score(final_bigram_counts,y_train,MultinomialNB,count_vect)
get_auc_score(final_tf_idf,y_train,MultinomialNB,tf_idf_vect)

For countvectorizer:
AUC: for  <class 'sklearn.naive_bayes.MultinomialNB'>   0.9458042036580849
For tf_idf
AUC: for  <class 'sklearn.naive_bayes.MultinomialNB'>   0.9521927828580388


In [36]:
get_auc_score(final_bigram_counts,y_train,RandomForestClassifier,count_vect)
get_auc_score(final_tf_idf,y_train,RandomForestClassifier,tf_idf_vect)

For countvectorizer:
AUC: for  <class 'sklearn.ensemble._forest.RandomForestClassifier'>   0.9353022262062695
For tf_idf
AUC: for  <class 'sklearn.ensemble._forest.RandomForestClassifier'>   0.9442017522092268


In [37]:
get_auc_score(final_bigram_counts,y_train,XGBClassifier,count_vect)
get_auc_score(final_tf_idf,y_train,XGBClassifier,tf_idf_vect)

For countvectorizer:
AUC: for  <class 'xgboost.sklearn.XGBClassifier'>   0.95217905563386
For tf_idf
AUC: for  <class 'xgboost.sklearn.XGBClassifier'>   0.9508926899201725


In [38]:
# Logistic Regression yields best result so far.

In [39]:
#SCORE ON TEST DATA

In [40]:
lr_test=LogisticRegression()
lr_test.fit(final_tf_idf,y_train)
roc_auc_score(lr_test.predict(test_tf),y_test)

0.9146401770055874

In [41]:
#Now lets fit the model on the whole dataset

In [42]:
tf_idf_final = TfidfVectorizer(ngram_range=(1,2),max_features=5000)
X=tf_idf_final.fit_transform(final['Text'])
y=final['Score']

In [43]:
lr_final=LogisticRegression()
lr_final.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [44]:
feature_names = np.array(tf_idf_final.get_feature_names())
sorted_coef_index = lr_final.coef_[0].argsort()
print('Smallest Coef: \n{}\n'.format(feature_names[sorted_coef_index-1][:10]))
print('Largest Coef: \n{}\n'.format(feature_names[sorted_coef_index-1][:-11:-1]))

Smallest Coef: 
['nose' 'disappoint' 'not going' 'worse' 'disappointed product'
 'awesome product' 'term' 'hoping' 'not worry' 'yuban']

Largest Coef: 
['greasy' 'besides' 'lot money' 'gone' 'delicate' 'example' 'per serving'
 'next time' 'yum yum' 'wonder']



In [45]:
# Our new model has correctly predicted bigrams like “not going”, “disappointed product” to negative reviews, 
# and bigrams like “delicate”, “yum yum” to positive reviews.
# Let’s test our new model:

In [47]:
print(lr_final.predict(tf_idf_final.transform(['The candy is not good, I would never buy them again','The candy is not bad, I will buy them again'])))

[0 1]


In [48]:
# Our latest model now correctly identifies them as negative and positive reviews respectively.